<a href="https://colab.research.google.com/github/dev-nileshpawar/python-aiml/blob/main/nova_multimodal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
!pip install boto3
import boto3
import json
import base64

In [29]:
from google.colab import userdata

PRO_MODEL_ID = "amazon.nova-pro-v1:0"
LITE_MODEL_ID = "amazon.nova-lite-v1:0"
MACRO_MODEL_ID = "amazon.nova-macro-v1:0"

YOUR_ACCESS_KEY = userdata.get('YOUR_ACCESS_KEY')
YOUR_SECRET_KEY = userdata.get("YOUR_SECRET_KEY")
client = boto3.client(
    service_name="bedrock-runtime",
    region_name="us-east-1",
    aws_access_key_id=YOUR_ACCESS_KEY,
    aws_secret_access_key=YOUR_SECRET_KEY,
)



In [30]:
from datetime import datetime

system_list = [{
    "text":"Act as a creative writing assistant. When the use provides you with topic, write a short story about that topic."
}]

message_list= [{"role": "user", "content":[{"text": "camping trip"}]}]

inf_params = {"max_new_tokens": 200, "top_p":0.9, "top_k":20}

request_body = {
    "messages": message_list,
    "system": system_list,
    "inferenceConfig": inf_params
}

start_time = datetime.now()

response = client.invoke_model_with_response_stream(modelId = LITE_MODEL_ID, body=json.dumps(request_body))

chunk_count = 0
time_to_first_token = None


stream = response.get('body')
if stream:
  for event in stream:
    chunk = event.get('chunk')
    if chunk:
      # print the response chunk
      chunk_json = json.loads(chunk.get('bytes').decode())
      # pretty print json
      content_block_delta = chunk_json.get('contentBlockDelta')
      if content_block_delta:
        if time_to_first_token is None:
          time_to_first_token = datetime.now() - start_time
          print(f"Time to first token: {time_to_first_token.total_seconds()} seconds")

        chunk_count+=1
        current_time = datetime.now()
        elapsed_time = current_time - start_time
        # print(f"Time elapsed: {elapsed_time.total_seconds()} seconds")
        print(content_block_delta.get('delta').get("text"), end="")
        # print(f"Total Chunks: {chunk_count}")

else:
  print("No response received")






Time to first token: 0.901164 seconds
The sun hung low in the sky, casting a golden hue over the dense forest as the Johnson family arrived at Pinewood Campground. Excitement buzzed in the air, mingling with the scent of pine and the distant sound of a babbling brook. 

"Alright, everyone! Let's set up camp!" Dad called, his voice filled with enthusiasm. 

Mom began unpacking the tent, while little sister Mia dashed off to explore the surroundings. She giggled as she skipped over fallen logs and peered into the underbrush, her imagination running wild with thoughts of fairies and hidden treasures.

Dad and older brother Jake took charge of gathering firewood, their laughter echoing through the trees as they stacked branches into a neat pile. The crackling fire soon roared to life, sending sparks dancing into the twilight.

As the stars began to twinkle overhead, the family gathered around the campfire. Dad told stories of his own childhood camping trips, his eyes twinkling with nostalg

# **Synchronous API**

In [31]:
system_list = [{"text": "You should respond all messages in French"}]

message_list = [{"role": "user", "content":[{"text":"Freecodecamp is a great platform to learn coding and technology in the field of computer science"}]}]

inf_params = {"max_new_tokens": 300, "top_p":0.9, "top_k":20}

native_request = {
    "messages": message_list,
    "system": system_list,
    "inferenceConfig": inf_params
}

response = client.invoke_model(modelId=LITE_MODEL_ID, body=json.dumps(native_request))

request_id = response["ResponseMetadata"]["RequestId"]
print(f"Request ID: {request_id}")

model_response = json.loads(response["body"].read())
print(json.dumps(model_response, indent=2))

Request ID: fed76cb0-affa-42f0-8d84-eef0bc778b97
{
  "output": {
    "message": {
      "content": [
        {
          "text": "Freecodecamp est une excellente plateforme pour apprendre la programmation et la technologie dans le domaine de l'informatique. Elle offre une vari\u00e9t\u00e9 de cours gratuits et interactifs qui couvrent divers langages de programmation, frameworks et technologies. Que vous soyez d\u00e9butant ou que vous cherchiez \u00e0 approfondir vos comp\u00e9tences, Freecodecamp propose des ressources p\u00e9dagogiques adapt\u00e9es \u00e0 tous les niveaux. De plus, la communaut\u00e9 active et les d\u00e9fis pratiques aident \u00e0 consolider les connaissances acquises."
        }
      ],
      "role": "assistant"
    }
  },
  "stopReason": "end_turn",
  "usage": {
    "inputTokens": 25,
    "outputTokens": 118,
    "totalTokens": 143,
    "cacheReadInputTokenCount": 0,
    "cacheWriteInputTokenCount": 0
  }
}


# **Streaming API Call**

In [32]:
from datetime import datetime

system_list = [{
    "text": "Act as a creative assistannt. When the user provides you with a topic, write a short story about the topic."
}]

message_list = [{
    "role":"user", "content" :[{"text": "A camping trip"}]
}]

inf_params = {"max_new_tokens": 200, "top_p": 0.9, "top_k": 20}

request_body = {
    "messages":message_list,
    "system": system_list,
    "inferenceConfig": inf_params
}

start_time = datetime.now()

response = client.invoke_model_with_response_stream(
    modelId=LITE_MODEL_ID, body=json.dumps(request_body)
)

chunk_count = 0;

time_to_first_token = None

stream = response.get("body")
if stream:
  for event in stream:
    chunk = event.get("chunk")
    if chunk:
      chunk_json = json.loads(chunk.get("bytes").decode())
      content_block_delta = chunk_json.get("contentBlockDelta")
      if content_block_delta:
        if time_to_first_token is None:
          time_to_first_token = datetime.now() - start_time
          print(f"Time to first token: {time_to_first_token.total_seconds()} seconds")
        else:
          print(content_block_delta.get("delta").get("text"), end="")
  print(f"\n\n Total chunks: {chunk_count}")

else:
  print("No response stream received")


Time to first token: 0.402233 seconds
 sun was just beginning to rise over the horizon, casting a warm golden glow over the forest as a group of friends set out on their camping trip. They had been planning this adventure for weeks, and now that the day had finally arrived, excitement buzzed in the air.

At the forefront of the group was Jake, the self-appointed leader of the expedition. He had packed the camping gear meticulously, ensuring that they had everything they needed for a comfortable stay in the great outdoors. His friends admired his thoroughness, knowing that his preparation would make the trip all the more enjoyable.

As they hiked deeper into the forest, the sounds of civilization faded away, replaced by the symphony of nature. Birds chirped melodiously, and the rustling of leaves underfoot created a soothing backdrop to their journey. The friends couldn't help but feel a sense of camaraderie as they walked side by side, sharing stories and laughter.

After a couple of h

# Image Understanding

Amazon Nova model allows to include multiple images in the payload with a max size of 25 MB, Amazon Nova models can analyze the passed images and answer questions, classify an image as well as summarize images based on provided instructions.

## Image size information
To provide the best possible results, amazon Nova automatically rescales input images up or down depending on their apsect ratio and original resolution, for each image, amazon nova first identifies aspect ratio first from 1:1, 1:2, 1:3, 1:4, 1:5, 1:6, 1:7, 1:8, 1:9, 2:3, 2:4 and their transposes. Then the image is rescaled so that at least one side of the image is greater than 896px or the length of shorter side of the original image, while maintaining the closest apsect ratio, there is a max resoultion 8000x8000 pixels.

| Image resolution | 900 x 450 | 900 x 900 | 1400 x 900 | 1.8k x 900 | 1.3k x 1.3k |
|--------|--------|-------- |-------- |-------- |-------- |
| Estimated token count | ~800 | ~1300 | ~1800 | ~2400 | ~2600 |


In [33]:
with open("./Snow-Transparent-File.png", "rb") as snow_image_file:
  snow_image_bytes = snow_image_file.read()
  base_64_encoded_data = base64.b64encode(snow_image_bytes)
  snow_base64_string = base_64_encoded_data.decode("utf-8")

with open("./desert.jpg", "rb") as desert_image_file:
  desert_image_bytes = desert_image_file.read()
  desert_base_64_encoded_data = base64.b64encode(desert_image_bytes)
  desert_base64_string = desert_base_64_encoded_data.decode("utf-8")


  system_list = [{
    "text": "You are an expter artist, when the user provides you with an image(s), Generate 3 separate art titles for EACH image individually. Clearly mention which title belongs to which image."
  }]
  message_list = [{
      "role": "user",
      "content": [{
          "image" : {
              "format": "png",
              "source": {
                  "bytes": snow_base64_string
              }
          },
        }, {
            "image": {
                "format": "jpg",
                "source": {
                    "bytes" : desert_base64_string
                }
            }
        },
        {"text":"Provide an art title for all these image(s)"}
      ]
  }]

  inf_params = {"max_new_tokens": 300, "top_p": 0.1, "top_k": 20, "temperature": 0.3}

  native_request = {
      "messages": message_list,
      "system": system_list,
      "inferenceConfig": inf_params
  }

  response = client.invoke_model(
      modelId=PRO_MODEL_ID, body=json.dumps(native_request)
  )

  model_response = json.loads(response["body"].read())
  print(model_response["output"]["message"]["content"][0]["text"])

**Image 1: Snowy Landscape**
1. "Winter's Embrace"
2. "Frosted Pines"
3. "Snowfall Symphony"

**Image 2: Desert Palms**
1. "Oasis Mirage"
2. "Desert Solitude"
3. "Golden Dunes"


# **Video Understanding**

In [36]:
with open("./mov_bbb.mp4", "rb") as video_file:
  video_bytes = video_file.read()
  video_base64_encoded_data = base64.b64encode(video_bytes)
  video_base64_string = video_base64_encoded_data.decode("utf-8")

system_list = [{
    "text": "You are an expert media analyst. When the user provides you with a video, provide a summary of the video."
}]

message_list = [{
    "role": "user",
    "content":[{
        "video": {
            "format": "mp4",
            "source": {
                "bytes": video_base64_string
            }
        }
    }, {
        "text": "Provide summary for this clip"
    }]
}]

inf_params = {"max_new_tokens": 500, "top_p": 0.1, "top_k": 20, "temperature": 0.3}


native_request = {
    "messages": message_list,
    "system": system_list,
    "inferenceConfig": inf_params
}

response = client.invoke_model(modelId=PRO_MODEL_ID, body=json.dumps(native_request))

model_response = json.loads(response["body"].read())
print(model_response["output"]["message"]["content"][0]["text"])



The video features a white rabbit in a grassy field. Initially, the rabbit is standing still, and a butterfly flies around it. Then, the rabbit starts running, and the butterfly continues to fly around it. Eventually, the rabbit stops running and stands still again in the grassy field.


# Fetch Video from S3

In [37]:
system_list = [{
    "text": "You are an expert media analyst. When the user provides you with a video, provide a summary of the video."
}]

message_list = [{
    "role": "user",
    "content":[{
        "video": {
            "format": "mp4",
            "source": {
                "s3Location": {
                    "uri": "s3://npawar868bucketupload/mov_bbb.mp4"
                }
            }
        }
    }, {
        "text": "Provide summary for this clip"
    }]
}]

inf_params = {"max_new_tokens": 500, "top_p": 0.1, "top_k": 20, "temperature": 0.3}


native_request = {
    "messages": message_list,
    "system": system_list,
    "inferenceConfig": inf_params
}

response = client.invoke_model(modelId=PRO_MODEL_ID, body=json.dumps(native_request))

model_response = json.loads(response["body"].read())
print(model_response["output"]["message"]["content"][0]["text"])



The video features a white rabbit in a grassy field. Initially, the rabbit is standing still, and a butterfly flies around it. Then, the rabbit starts running, and the butterfly continues to fly around it. Eventually, the rabbit stops running and stands still again in the grassy field.
